In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np

In [2]:
## load the model
model = load_model('model.h5')

# loading encoders and scaler
with open('OneHotEncoder_Geography.pkl', 'rb') as file:
  OneHotEncoder_Geography = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
  label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
  scaler = pickle.load(file)

In [3]:
# example input data
input_data = {
    'CreditScore': 650,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 65000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 70000
}

In [4]:
# encode geography
encode_geo = OneHotEncoder_Geography.transform([[input_data['Geography']]]).toarray()
geo_encoder_df = pd.DataFrame(encode_geo, columns=OneHotEncoder_Geography.get_feature_names_out(['Geography']))

C:\Users\ayush\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [5]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_data_df = pd.DataFrame([input_data])
input_data_df = pd.concat([input_data_df.reset_index(drop=True), geo_encoder_df], axis=1)
input_data_df = input_data_df.drop('Geography', axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,Male,40,3,65000,2,1,1,70000,1.0,0.0,0.0


In [7]:
# Encode Gender
input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,1,40,3,65000,2,1,1,70000,1.0,0.0,0.0


In [8]:
# scaling the input data
input_data_scale = scaler.transform(input_data_df)
input_data_scale

array([[-0.01709861,  0.91324755,  0.10479359, -0.69539349, -0.17775624,
         0.80843615,  0.64920267,  0.97481699, -0.5290988 ,  1.00150113,
        -0.57946723, -0.57638802]])

In [9]:
# Predict the data
prediction = model.predict(input_data_scale)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step


array([[0.02745329]], dtype=float32)

In [10]:
prediction_probability = prediction[0][0]
prediction_probability

0.027453287

In [11]:
if prediction_probability > 0.5:
  print("Customer is likely to churn.")
else:
  print("Customer is not likely to churn.")

Customer is not likely to churn.
